- Your architecture should consist of a single hidden layer with up to k nodes.
- You can use any activation function (e.g., sigmoid, tanh, etc.) in the hidden nodes.
- Your model must use a bias term at the input and hidden layers. It can be a standalone term or
be incorporated in the weight matrices.
- You should use gradient descent to train your FFNN.
- You may find it helpful to use random number seeds for reproducibility when debugging.
- You do not need to use a GPU for this assignment, and your models should train in less than one
minute each.
- You are responsible for selecting hyperparameters (e.g., number of hidden nodes, learning rate,
training epochs, batch sizes, early stopping criteria, lambda, etc.). The goal is to get “good”
performance from your model, but an exhaustive hyper-parameter search is unnecessary.
- All code, exhibits and answers to free-response questions must be in a single Jupyter notebook.
- Your code should use parameters to control all functionality needed to complete specific tasks
(see below).

In [ ]:
import torch

torch.manual_seed(42)

In [ ]:
'''
Define model
'''

import torch
import torch.nn as nn
import torch.nn.functional as F

class SingleLayerFFNN(nn.Module):
    def __init__(self, hidden_size, activation_fn="sigmoid"):
        super(SingleLayerFFNN, self).__init__()

        self.input_layer = nn.Linear(2, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, 2, bias=True)

        if activation_fn == "sigmoid":
            self.activation = torch.sigmoid
        elif activation_fn == "tanh":
            self.activation = torch.tanh
        elif activation_fn == "relu":
            self.activation = torch.relu

    def forward(self, x):
        # One hidden layer
        hidden = self.activation(self.input_layer(x))
        output = self.output_layer(hidden)

        return output

In [ ]:
'''
Train model
'''

import torch.optim as optim

def train_model(model, train_loader, num_epochs, learning_rate):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        # loss for this epoch
        epoch_loss = 0
        for data, target in train_loader:
            target = target.float().unsqueeze(1)
            optimizer.zero_grad() # reset gradients
            output = model(data) # forward pass
            # print(output, target)
            loss = criterion(output, target) # calculate loss
 
            loss.backward() # backpropagation
            optimizer.step() # take step based on gradients and lr

            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

In [ ]:
'''
Take in raw csv file path and batch size, return DataLoader object
'''

import csv
import torch
from torch.utils.data import DataLoader, TensorDataset

def load_data(file_path, batch_size):
    inputs = []
    labels = []

    with open(file_path, "r") as csvfile:
        reader = csv.reader(csvfile)
        next(reader) # skip header row
        for row in reader:
            labels.append(float(row[0]))
            inputs.append([float(row[1]), float(row[2])])

    inputs_tensor = torch.tensor(inputs, dtype=torch.float32)
    labels_tensor = torch.tensor(labels, dtype=torch.float32)

    dataset = TensorDataset(inputs_tensor, labels_tensor)
    loader = DataLoader(dataset, batch_size=batch_size)

    return loader


In [ ]:
'''
Train model with train set. 
Hyperparameters: hidden_size, batch_size, num_epochs, learning_rate
'''

file_path = "spiral_train.csv"

# hidden_size from [2, 3, 5, 7, 9] 
hidden_size = 7
batch_size = 8
# activation from "sigmoid", "relu", "tanh"
activation_fn = "relu"
num_epochs = 300
learning_rate = 0.1

train_loader = load_data(file_path, batch_size)
model = SingleLayerFFNN(hidden_size, activation_fn)

train_model(model, train_loader, num_epochs, learning_rate)

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            # between the probabilities for 0 and 1, pick higher
            preds = torch.argmax(output, dim=1)
            all_preds.extend(preds.numpy())
            all_targets.extend(target.numpy())

    all_preds = torch.tensor(all_preds).squeeze()
    all_targets = torch.tensor(all_targets)
    
    correct = (all_preds == all_targets).sum().item()
    total = len(all_targets)
    accuracy = correct / total
    
    print(f"Accuracy: {accuracy:.4f}")
    return accuracy

In [ ]:
eval_file_path = "spiral_test.csv"
eval_batch_size = 4

eval_loader = load_data(eval_file_path, eval_batch_size)
evaluate_model(model, eval_loader)

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_curves(model, train_loader, val_loader, num_epochs, optimizer, criterion):
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        # Training loop
        model.train()
        running_train_loss = 0.0
        for data, target in train_loader:
            target = target.squeeze()
            optimizer.zero_grad()
            output = model(data).squeeze()
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()
        train_loss = running_train_loss / len(train_loader)
        train_losses.append(train_loss)
        
        # Validation loop
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for data, target in val_loader:
                target = target.squeeze()
                output = model(data).squeeze()
                loss = criterion(output, target)
                running_val_loss += loss.item()
        val_loss = running_val_loss / len(val_loader)
        val_losses.append(val_loss)
        
        print(f"Epoch {epoch + 1}/{num_epochs} - Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
    
    # Plotting the loss curves
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss')
    plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss Curves')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return train_losses, val_losses


In [ ]:
file_path = "spiral_train.csv"

hidden_size = 7
batch_size = 8
activation_fn = "relu"
num_epochs = 300
learning_rate = 0.1

eval_file_path = "spiral_valid.csv"
eval_batch_size = 4


train_loader = load_data(file_path, batch_size)
model = SingleLayerFFNN(hidden_size, activation_fn)
eval_loader = load_data(eval_file_path, eval_batch_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

plot_loss_curves(model, train_loader, eval_loader, num_epochs, optimizer, criterion)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D


def plot_decision_surface(model, test_loader, feature_range=(-2, 2), resolution=0.01):
    """
    Plots the decision surface of the model along with the test set observations.
    
    Args:
        model: Trained PyTorch model.
        test_loader: DataLoader for the test set.
        feature_range: Tuple (min, max) for the grid of features.
        resolution: Step size for the grid.
    """
    # Generate a grid of points
    x_min, x_max = feature_range
    y_min, y_max = feature_range
    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, resolution),
        np.arange(y_min, y_max, resolution)
    )
    
    # Flatten the grid to feed into the model
    grid_points = np.c_[xx.ravel(), yy.ravel()]
    grid_points_tensor = torch.tensor(grid_points, dtype=torch.float32)
    
    # Get model predictions on the grid
    model.eval()
    with torch.no_grad():
        predictions = model(grid_points_tensor)
        print(predictions)
        predicted_labels = torch.argmax(predictions, axis=1).numpy()
        print(predicted_labels)
    
    # Reshape predictions back to the grid shape
    Z = predicted_labels.reshape(xx.shape)
    
    # Plot decision surface
    plt.figure(figsize=(10, 6))
    plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.coolwarm)
    
    # Plot test data points
    for data, target in test_loader:
        data = data.numpy()
        target = target.numpy()
        plt.scatter(data[:, 0], data[:, 1], c=target, edgecolor='k', cmap=plt.cm.coolwarm, s=50, label='Test Data')
    
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='Class 0',
               markerfacecolor='red', markersize=10),
        Line2D([0], [0], marker='o', color='w', label='Class 1',
               markerfacecolor='blue', markersize=10)
    ]

    plt.title('Learned Decision Surface')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.legend(handles=legend_elements, loc='upper right')
    plt.grid(True)
    plt.show()


In [ ]:
eval_file_path = "spiral_test.csv"
eval_batch_size = 4

test_loader = load_data(eval_file_path, eval_batch_size)
plot_decision_surface(model, test_loader, feature_range=(-20, 20), resolution=0.01)